In [1]:
import torch

In [37]:
pred = torch.tensor([[0.81, 0.8, 0.7, 0.6, 0.5]])
truth = torch.tensor([[5, 4, 3, 2, 1]], dtype=torch.float32)-1

In [38]:
pred.argsort(dim=1).float()

tensor([[4., 3., 2., 1., 0.]])

In [39]:
loss = torch.nn.functional.cross_entropy(pred.argsort(dim=1).float().softmax(dim=1), truth.softmax(dim=1))
loss

tensor(1.3702)

In [40]:
pred.argsort(dim=1).float().softmax(dim=1), truth.softmax(dim=1)

(tensor([[0.6364, 0.2341, 0.0861, 0.0317, 0.0117]]),
 tensor([[0.6364, 0.2341, 0.0861, 0.0317, 0.0117]]))

In [41]:
# ListNet loss introduced in "Learning to Rank: From Pairwise Approach to Listwise Approach".
preds_smax = pred.argsort(dim=1).float().softmax(dim=1)# + eps
preds_log = torch.log(preds_smax)

torch.mean(-torch.sum(truth.softmax(dim=1) * preds_log, dim=1))

tensor(1.0000)